# Lesson 6.6.1 多分类评估指标的macro与weighted过程

&emsp;&emsp;在正式讨论关于网格搜索的进阶使用方法之前，我们需要先补充一些关于多分类问题的评估指标计算过程。在此前的课程中，我们曾经介绍过分类模型在解决多分类问题时的不同策略，同时也介绍过二分类问题的更高级评估指标，如f1-score和roc-auc等，接下来我们将详细讨论关于多分类预测结果在f1-socre和roc-auc中的评估过程，以及在sklearn中如何调用函数进行计算。

In [95]:
# 科学计算模块
import numpy as np
import pandas as pd

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# 自定义模块
from ML_basic_function import *

# Scikit-Learn相关模块
# 评估器类
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# 实用函数
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 数据准备
from sklearn.datasets import load_iris

- 多分类F1-Score评估指标

&emsp;&emsp;首先导入和F1-Score相关的评估指标计算函数

In [2]:
from sklearn.metrics import precision_score,recall_score,f1_score

然后简单查看相关说明文档，发现这几组和混淆矩阵相关的评估指标基本是共用了一套参数命名，并且大多数参数其实都是作用于多分类问题，对于二分类问题，我们可以简单调用相关函数直接计算：

In [28]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([1, 1, 0, 1, 0, 1])

In [29]:
precision_score(y_true, y_pred), recall_score(y_true, y_pred), f1_score(y_true, y_pred)

(0.75, 1.0, 0.8571428571428571)

In [3]:
precision_score?

Signature:
precision_score(
    y_true,
    y_pred,
    *,
    labels=None,
    pos_label=1,
    average='binary',
    sample_weight=None,
    zero_division='warn',
)
Docstring:
Compute the precision

The precision is the ratio ``tp / (tp + fp)`` where ``tp`` is the number of
true positives and ``fp`` the number of false positives. The precision is
intuitively the ability of the classifier not to label as positive a sample
that is negative.

The best value is 1 and the worst value is 0.

Read more in the :ref:`User Guide <precision_recall_f_measure_metrics>`.

Parameters
----------
y_true : 1d array-like, or label indicator array / sparse matrix
    Ground truth (correct) target values.

y_pred : 1d array-like, or label indicator array / sparse matrix
    Estimated targets as returned by a classifier.

labels : list, optional
    The set of labels to include when ``average != 'binary'``, and their
    order if ``average is None``. Labels present in the data can be
    excluded, for exa

具体参数含义解释如下：

|Name|Description|      
|:--:|:--:| 
|y_true|数据集真实标签| 
|y_pred|标签预测结果|
|labels|允许以列表形式输入其他形态的标签，一般不进行修改|
|pos_label|positive类别标签|
|average|多分类时指标计算方法|
|sample_weight|不同类别的样本权重|
|zero_division|当分母为0时返回结果|

其中，需要重点介绍多分类问题时average参数不同取值时的计算方法。此处以recall为例进行计算，重点介绍当average取值为'macro'、'micro'和'weighted'的情况，其他指标也类似，例如有简单多分类问题如下：

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gsgjd36uzbj30ry0pm0xi.jpg" alt="1" style="zoom:30%;" /></center>

我们令1类标签为0、2类标签为1、3类标签为2，则上述数据集真实标签为：

In [31]:
y_true = np.array([0, 1, 2, 2, 0, 1, 1, 2, 0, 2])

并且最终分类预测结果为：

In [32]:
y_pred = np.array([0, 1, 0, 2, 2, 1, 2, 2, 0, 2])

据此我们可以构造多分类混淆矩阵如下：

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gsgjmmgd2dj314m0b676i.jpg" alt="1" style="zoom:30%;" /></center>

据此我们可以计算三个类别的TP和FN：

In [33]:
tp1 = 2
tp2 = 2
tp3 = 3

In [34]:
fn1 = 1
fn2 = 1
fn3 = 1

接下来有两种计算recall的方法，其一是先计算每个类别的recall，然后求均值：

In [35]:
re1 = 2/3
re2 = 2/3
re3 = 3/4

In [36]:
np.mean([re1, re2, re3])

0.6944444444444443

这也就是average参数取值为macro时的计算结果：

In [37]:
recall_score(y_true, y_pred, average='macro')

0.6944444444444443

当然，如果上述手动实现过程不求均值，而是根据每个类别的数量进行加权求和，则就是参数average参数取值为weighted时的结果：

In [38]:
re1 * 3/10 + re2 * 3/10 + re3 * 4/10

0.7

In [39]:
recall_score(y_true, y_pred, average='weighted')

0.7

当然，还有另外一种计算方法，那就是先计算整体的TP和FN，然后根据整体TP和FN计算recall：

In [40]:
tp = tp1 + tp2 + tp3
fn = fn1 + fn2 + fn3

In [41]:
tp / (tp+fn)

0.7

该过程也就是average参数取值micro时的计算结果：

In [42]:
recall_score(y_true, y_pred, average='micro')

0.7

&emsp;&emsp;对于上述三个不同参数的选取，首先如果是样本不平衡问题（如果是要侧重训练模型判别小类样本的能力的情况下）、则应排除weighted参数，以避免赋予大类样本更高的权重。除此以外，在大多数情况下这三个不同的参数其实并不会对最后评估器的选取结果造成太大影响，只是在很多要求严谨的场合下需要说明多分类的评估结果的计算过程，此时需要简单标注下是按照何种方法进行的计算。        
&emsp;&emsp;不过，如果是混淆矩阵中相关指标和roc-auc指标放在一起讨论，由于新版sklearn中roc-auc本身不支持在多分类时按照micro计算、只支持macro计算，因此建议混淆矩阵的多分类计算过程也选择macro过程，以保持一致。后续在没有进行其他特殊说明的情况下，课上统一采用macro指标进行多分类问题评估指标的计算。

> 不过值得注意的是，还有一种观点，尽管micro和macro方法在混淆矩阵相关指标的计算过程中差别不大，在roc-auc中，macro指标并不利于非平衡样本的计算（混淆矩阵中可以通过positive的类别选择来解决这一问题），需要配合ovr分类方法才能够有所改善。

- 多分类ROC-AUC评估指标

&emsp;&emsp;接下来继续讨论关于多分类的ROC-AUC评估指标的相关问题：

In [43]:
from sklearn.metrics import roc_auc_score

能够发现，roc_auc_score评估指标函数中大多数参数都和此前介绍的混淆矩阵中评估指标类似。接下来我们简单尝试使用roc-auc函数进行评估指标计算，根据roc-auc的计算流程可知，此处我们需要在y_pred参数位中输入模型概率预测结果：

In [44]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([0.9, 0.7, 0.2, 0.7, 0.4, 0.8])

In [45]:
roc_auc_score(y_true, y_pred)

0.9444444444444444

当然，如果我们在y_pred参数中输入分类结果，该函数也能计算出最终结果：

In [46]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([1, 1, 0, 1, 0, 1])

In [47]:
roc_auc_score(y_true, y_pred)

0.8333333333333334

不过，此时模型会默认预测标签为0的概率结果为0.4、预测标签为1的概率预测结果为0.6，即上述结果等价于：

In [48]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([0.6, 0.6, 0.4, 0.6, 0.4, 0.6])

In [49]:
roc_auc_score(y_true, y_pred)

0.8333333333333334

> 即计算过程会默认模型概率预测结果更差。

接下来详细解释roc-auc中其他参数：

In [50]:
roc_auc_score?

Signature:
roc_auc_score(
    y_true,
    y_score,
    *,
    average='macro',
    sample_weight=None,
    max_fpr=None,
    multi_class='raise',
    labels=None,
)
Docstring:
Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC)
from prediction scores.

Note: this implementation can be used with binary, multiclass and
multilabel classification, but some restrictions apply (see Parameters).

Read more in the :ref:`User Guide <roc_metrics>`.

Parameters
----------
y_true : array-like of shape (n_samples,) or (n_samples, n_classes)
    True labels or binary label indicators. The binary and multiclass cases
    expect labels with shape (n_samples,) while the multilabel case expects
    binary label indicators with shape (n_samples, n_classes).

y_score : array-like of shape (n_samples,) or (n_samples, n_classes)
    Target scores. In the binary and multilabel cases, these can be either
    probability estimates or non-thresholded decision values (as returned
    by `dec

|Name|Description|      
|:--:|:--:| 
|max_fpr|fpr最大值，fpr是roc曲线的横坐标| 
|multi_class|分类器在进行多分类时进行的多分类问题处理策略|

&emsp;&emsp;此处需要注意的是关于multi_class参数的选择。一般来说sklearn中的multi_class参数都是二分类器中用于解决多元分类问题时的参数（如逻辑回归），而由于roc-auc需要分类结果中的概率来完成最终计算，因此需要知道概率结果对应分类标签——即到底是以ovo还是ovr模式在进行多分类，因此如果是进行多分类roc-auc计算时，需要对其进行明确说明。       
&emsp;&emsp;不过对于多分类逻辑回归来说，无论是ovr还是mvm策略，最终分类结果其实都可以看成是ovr分类结果，因此如果是多分类逻辑回归计算roc-auc，需要设置multi_class参数为ovr。同时由于根据roc-auc的函数参数说明可知，在multi_class参数取为ovr时，average参数取值为macro时能够保持一个较高的偏态样本敏感性，因此对于roc-auc来说，大多数时候average参数建议取值为macro。总结一下，对于roc-auc进行多分类问题评估时，建议选择的参数组合是ovr/ovo+macro，而ovr/ovo的参数选择需要根据具体的多分类模型来定，如果是围绕逻辑回归多分类评估器来进行结果评估，则建议roc-auc和逻辑回归评估器的multi_class参数都选择ovr。

> 在新版的sklearn中，roc-auc函数的multi_class参数已不支持micro参数，面对多分类问题，该参数只能够在macro和weighted中进行选择。

接下来我们简单测算average参数中macro和weighted的计算过程。还是围绕上述数据集进行计算：

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gsgjd36uzbj30ry0pm0xi.jpg" alt="1" style="zoom:30%;" /></center>

据此我们可以计算每个类别单独的roc-auc值：

In [51]:
y_true_1 = np.array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0])
y_pred_1 = np.array([0.8, 0.2, 0.5, 0.2, 0.3, 0.1, 0.3, 0.3, 0.9, 0.3])

In [52]:
r1 = roc_auc_score(y_true_1, y_pred_1)
r1

0.8809523809523809

In [53]:
y_true_2 = np.array([0, 1, 0, 0, 0, 1, 1, 0, 0, 0])
y_pred_2 = np.array([0.2, 0.6, 0.3, 0, 0.2, 0.8, 0.2, 0.3, 0, 0.1])

In [54]:
r2 = roc_auc_score(y_true_2, y_pred_2)
r2

0.8571428571428571

In [55]:
y_true_3 = np.array([0, 0, 1, 1, 0, 0, 0, 1, 0, 1])
y_pred_3 = np.array([0, 0.2, 0.2, 0.8, 0.5, 0.1, 0.5, 0.4, 0.1, 0.6])

In [56]:
r3 = roc_auc_score(y_true_3, y_pred_3)
r3

0.8125

此时r1、r2、r3的均值如下：

In [57]:
np.mean([r1, r2, r3])

0.8501984126984127

该结果应当和macro+multi_class参数计算结果相同

In [58]:
y_pred = np.concatenate([y_pred_1.reshape(-1, 1), y_pred_2.reshape(-1, 1), y_pred_3.reshape(-1, 1)], 1)
y_pred

array([[0.8, 0.2, 0. ],
       [0.2, 0.6, 0.2],
       [0.5, 0.3, 0.2],
       [0.2, 0. , 0.8],
       [0.3, 0.2, 0.5],
       [0.1, 0.8, 0.1],
       [0.3, 0.2, 0.5],
       [0.3, 0.3, 0.4],
       [0.9, 0. , 0.1],
       [0.3, 0.1, 0.6]])

In [59]:
y_true = np.array([0, 1, 2, 2, 0, 1, 1, 2, 0, 2])

In [60]:
roc_auc_score(y_true, y_pred, average='macro', multi_class='ovr')

0.8501984126984127

当然，如果roc-auc函数的参数是ovr+weighted，则计算结果过程验证如下：

In [61]:
r1 * 3/10 + r2 * 3/10 + r3 * 4/10

0.8464285714285713

In [62]:
roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr')

0.8464285714285713

至此，我们就能够较为清楚的了解关于f1-score和roc-auc评估指标在调用sklearn中相关函数解决多分类问题评估的具体方法。